#Exploratory data analysis

In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import warnings
import time
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
warnings.filterwarnings('ignore')

# Import libraries for visualization and set default values.
import matplotlib.pyplot as plt
plt.style.use(['seaborn'])
sns.set_theme(style="whitegrid", palette=sns.color_palette("tab10"))

# 2. Load Dataset
---

In [ ]:
path='..\DATA.xlsx'
data_df=pd.read_excel(path, usecols=[""] )

In [ ]:
data_df.head()

In [ ]:
data_df.info()

# 3. Cleaning

In [ ]:
pre_categorical_data = ["AREA", "VARIEDAD", "EDAD", "Corte"]

In [ ]:
data_df[pre_categorical_data]

Some attributes have the wrong type or have typos. Let's fix them

In [ ]:
def replace_matches_in_column(df, column, string_to_match, min_ratio):
    # get a list of unique strings
    strings = df[column].unique()
    matches = process.extract(string_to_match, strings,
                                         limit=10, scorer=fuzz.token_sort_ratio)
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    rows_with_matches = df[column].isin(close_matches)
    df.loc[rows_with_matches, column] = string_to_match
    print("All done!")

In [ ]:
replace_matches_in_column(data_df, 'Corte', 'c', 90)
replace_matches_in_column(data_df, 'Corte', 'plant', 80)
data_df["Corte"] = data_df["Corte"].replace(['6d', 'ptlla', '2'],['6.0', 'Plantilla', '2.0'])

In [ ]:
data_df["EDAD"] = pd.to_numeric(data_df.EDAD.astype(str).apply(lambda x:"".join(i for i in x if i.isdigit() or i==".")), errors='coerce')
data_df["AREA"] = pd.to_numeric(data_df.AREA.astype(str).apply(lambda x:"".join(i for i in x if ~i.isdigit() or i=='.')), errors='coerce')

## Missing values

Let's check if there are missing values in the dataset

In [ ]:
missing_values = data_df.isna().sum() / len(scylv_df)
missing_values = missing_values[missing_values > 0]*100
missing_values.sort_values(inplace=True, ascending=False)
sns.barplot(x=missing_values, y=missing_values.index)
plt.title("Missing values in %")
plt.xlabel("Percentage %")
plt.ylabel("Features")


There are almost no missing values. We replace the missing values with the mean of the column or the mode of the column(categorical).

# 4. Exploratory data analysis


In [ ]:
corrMatrix = data_df.corr()
fig, ax = plt.subplots(figsize=(18,10))
sns.heatmap(corrMatrix, annot=True, fmt='.2f')

As we can see, SEM. and COM. are the opposite, so we can drop one of them.

In [ ]:
CrosstabResult=pd.crosstab(index=scylv_df[' SEM.'],columns=scylv_df['COM.'])
CrosstabResult.plot.bar()
data_df = data_df.drop([' SEM.'], axis=1)

Tm and Tmax are highly correlated so we can drop one of them.

In [ ]:
fig, ax = plt.subplots(figsize=(18,10), nrows=2)
ax[0].plot(data_df['Tmax'], label = 'Tmax')
ax[0].plot(data_df['Tm'], label = 'Tm')
ax[0].set_ylabel('Celsius degrees')
ax[0].legend()
ax[1].scatter(data_df['Tm'], scylv_df['Tmax'])
ax[1].set_xlabel('Tm')
ax[1].set_ylabel('Tmax')

data_df = data_df.drop(['Tmax'], axis=1)

Not enough correlation to drop any of them.

In [ ]:
plt.scatter(data_df['V_viento'], data_df['P_vaporw'])
plt.xlabel('V_viento')
plt.ylabel('P_vaporw')

In [ ]:
numerical = ['EDAD', 'NDVI', 'pH', 'Da', 'CO', 'Altitud', 'CIC', 'PPT', 'AREA',  'S_rad', 'Tm', 'Tmin', 'P_vaporw', 'V_viento', 'Arena', 'fragmentos', 'Arcilla', 'Nitrogeno']
f, axes = plt.subplots(6, 3, figsize=(20, 35))
colors = sns.color_palette("bright")
i=0
for ax, feature in zip(axes.flat, numerical):
    ax.grid(False)
    sns.kdeplot(data=scylv_df, x=feature, ax=ax, color= colors[i%10], label=feature)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.legend()
    i+=1


In [ ]:
data_df.to_csv('..\cleaned_data.csv', index=False)